### Connect to Data Lake

In [0]:
app_id = "Your_App_ID"
tenant_id = "Your_Tenant_ID"

configs = {"fs.azure.account.auth.type": "OAuth",
       "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
       "fs.azure.account.oauth2.client.id": app_id,
       "fs.azure.account.oauth2.client.secret": dbutils.secrets.get(scope = "hblscope", key ="hblkeyvaultsecretfordatabricks"),
       "fs.azure.account.oauth2.client.endpoint": "https://login.chinacloudapi.cn/"+tenant_id+"/oauth2/token",
       "fs.azure.createRemoteFileSystemDuringInitialization": "true"}

dbutils.fs.mount(
source = "abfss://hbldatalake@hbldatalake.dfs.core.chinacloudapi.cn/",
mount_point = "/mnt/az_datalake",
extra_configs = configs)

Out[1]: True

In [0]:
%fs ls /mnt/az_datalake/salesdata/

path name size dbfs:/mnt/az_datalake/salesdata/call_center/ call_center/ 0 dbfs:/mnt/az_datalake/salesdata/catalog_page/ catalog_page/ 0 dbfs:/mnt/az_datalake/salesdata/catalog_returns/ catalog_returns/ 0 dbfs:/mnt/az_datalake/salesdata/catalog_sales/ catalog_sales/ 0 dbfs:/mnt/az_datalake/salesdata/customer/ customer/ 0 dbfs:/mnt/az_datalake/salesdata/customer_address/ customer_address/ 0 dbfs:/mnt/az_datalake/salesdata/customer_demographics/ customer_demographics/ 0 dbfs:/mnt/az_datalake/salesdata/date_dim/ date_dim/ 0 dbfs:/mnt/az_datalake/salesdata/household_demographics/ household_demographics/ 0 dbfs:/mnt/az_datalake/salesdata/income_band/ income_band/ 0 dbfs:/mnt/az_datalake/salesdata/inventory/ inventory/ 0 dbfs:/mnt/az_datalake/salesdata/item/ item/ 0 dbfs:/mnt/az_datalake/salesdata/promotion/ promotion/ 0 dbfs:/mnt/az_datalake/salesdata/reason/ reason/ 0 dbfs:/mnt/az_datalake/salesdata/ship_mode/ ship_mode/ 0 dbfs:/mnt/az_datalake/salesdata/store/ store/ 0 dbfs:/mnt/az_datalake/salesdata/store_returns/ store_returns/ 0 dbfs:/mnt/az_datalake/salesdata/store_sales/ store_sales/ 0 dbfs:/mnt/az_datalake/salesdata/time_dim/ time_dim/ 0 dbfs:/mnt/az_datalake/salesdata/warehouse/ warehouse/ 0 dbfs:/mnt/az_datalake/salesdata/web_page/ web_page/ 0 dbfs:/mnt/az_datalake/salesdata/web_returns/ web_returns/ 0 dbfs:/mnt/az_datalake/salesdata/web_sales/ web_sales/ 0 dbfs:/mnt/az_datalake/salesdata/web_site/ web_site/ 0

### Data Exploration

In [0]:
customerSchema = '''customer_sk LONG,
customer_id string,
current_cdemo_sk LONG,
current_hdemo_sk LONG,
current_addr_sk LONG,
first_shipto_date_sk LONG,
first_sales_date_sk LONG,
salutation STRING,
firstName STRING,
lastName STRING,
preferred_Cust_Flag STRING,
birth_Day LONG,
birth_Month LONG,
birth_Year LONG,
birth_Country STRING,
login STRING,
emailAddress STRING,
last_review_date LONG
'''

In [0]:
customer_df = (spark.read
.option("sep","|")
.schema(customerSchema)
.option("inferSchema","true")
.csv("/mnt/az_datalake/salesdata/customer/customer_001.dat"))

In [0]:
customer_df.printSchema()

root
-- customer_sk: long (nullable = true)
-- customer_id: string (nullable = true)
-- current_cdemo_sk: long (nullable = true)
-- current_hdemo_sk: long (nullable = true)
-- current_addr_sk: long (nullable = true)
-- first_shipto_date_sk: long (nullable = true)
-- first_sales_date_sk: long (nullable = true)
-- salutation: string (nullable = true)
-- firstName: string (nullable = true)
-- lastName: string (nullable = true)
-- preferred_Cust_Flag: string (nullable = true)
-- birth_Day: long (nullable = true)
-- birth_Month: long (nullable = true)
-- birth_Year: long (nullable = true)
-- birth_Country: string (nullable = true)
-- login: string (nullable = true)
-- emailAddress: string (nullable = true)
-- last_review_date: long (nullable = true)

### Let us look at he Customer Demographics Dataset

In [0]:
customerDemographicsSchema = ''' cd_demo_sk long,
cd_gender string,
cd_marital_status string,
cd_education_status string,
cd_purchase_estimate string,
cd_credit_rating string,
cd_dep_count int,
cd_dep_employed_count int,
cd_dep_college_count int
'''

In [0]:
customer_demographics = spark.read.option("sep","|").schema(customerDemographicsSchema).csv("/mnt/az_datalake/salesdata/customer_demographics/customer_demographics_001.dat")

In [0]:
display(customer_demographics)

cd_demo_sk cd_gender cd_marital_status cd_education_status cd_purchase_estimate cd_credit_rating cd_dep_count cd_dep_employed_count cd_dep_college_count 1 M M Primary 500 Good 0 0 0 2 F M Primary 500 Good 0 0 0 3 M S Primary 500 Good 0 0 0 4 F S Primary 500 Good 0 0 0 5 M D Primary 500 Good 0 0 0 6 F D Primary 500 Good 0 0 0 7 M W Primary 500 Good 0 0 0 8 F W Primary 500 Good 0 0 0 9 M U Primary 500 Good 0 0 0 10 F U Primary 500 Good 0 0 0 11 M M Secondary 500 Good 0 0 0 12 F M Secondary 500 Good 0 0 0 13 M S Secondary 500 Good 0 0 0 14 F S Secondary 500 Good 0 0 0 15 M D Secondary 500 Good 0 0 0 16 F D Secondary 500 Good 0 0 0 17 M W Secondary 500 Good 0 0 0 18 F W Secondary 500 Good 0 0 0 19 M U Secondary 500 Good 0 0 0 20 F U Secondary 500 Good 0 0 0 21 M M College 500 Good 0 0 0 22 F M College 500 Good 0 0 0 23 M S College 500 Good 0 0 0 24 F S College 500 Good 0 0 0 25 M D College 500 Good 0 0 0 26 F D College 500 Good 0 0 0 27 M W College 500 Good 0 0 0 28 F W College 500 Good 0 0 0 29 M U College 500 Good 0 0 0 30 F U College 500 Good 0 0 0 31 M M 2 yr Degree 500 Good 0 0 0 32 F M 2 yr Degree 500 Good 0 0 0 33 M S 2 yr Degree 500 Good 0 0 0 34 F S 2 yr Degree 500 Good 0 0 0 35 M D 2 yr Degree 500 Good 0 0 0 36 F D 2 yr Degree 500 Good 0 0 0 37 M W 2 yr Degree 500 Good 0 0 0 38 F W 2 yr Degree 500 Good 0 0 0 39 M U 2 yr Degree 500 Good 0 0 0 40 F U 2 yr Degree 500 Good 0 0 0 41 M M 4 yr Degree 500 Good 0 0 0 42 F M 4 yr Degree 500 Good 0 0 0 43 M S 4 yr Degree 500 Good 0 0 0 44 F S 4 yr Degree 500 Good 0 0 0 45 M D 4 yr Degree 500 Good 0 0 0 46 F D 4 yr Degree 500 Good 0 0 0 47 M W 4 yr Degree 500 Good 0 0 0 48 F W 4 yr Degree 500 Good 0 0 0 49 M U 4 yr Degree 500 Good 0 0 0 50 F U 4 yr Degree 500 Good 0 0 0 51 M M Advanced Degree 500 Good 0 0 0 52 F M Advanced Degree 500 Good 0 0 0 53 M S Advanced Degree 500 Good 0 0 0 54 F S Advanced Degree 500 Good 0 0 0 55 M D Advanced Degree 500 Good 0 0 0 56 F D Advanced Degree 500 Good 0 0 0 57 M W Advanced Degree 500 Good 0 0 0 58 F W Advanced Degree 500 Good 0 0 0 59 M U Advanced Degree 500 Good 0 0 0 60 F U Advanced Degree 500 Good 0 0 0 61 M M Unknown 500 Good 0 0 0 62 F M Unknown 500 Good 0 0 0 63 M S Unknown 500 Good 0 0 0 64 F S Unknown 500 Good 0 0 0 65 M D Unknown 500 Good 0 0 0 66 F D Unknown 500 Good 0 0 0 67 M W Unknown 500 Good 0 0 0 68 F W Unknown 500 Good 0 0 0 69 M U Unknown 500 Good 0 0 0 70 F U Unknown 500 Good 0 0 0 71 M M Primary 1000 Good 0 0 0 72 F M Primary 1000 Good 0 0 0 73 M S Primary 1000 Good 0 0 0 74 F S Primary 1000 Good 0 0 0 75 M D Primary 1000 Good 0 0 0 76 F D Primary 1000 Good 0 0 0 77 M W Primary 1000 Good 0 0 0 78 F W Primary 1000 Good 0 0 0 79 M U Primary 1000 Good 0 0 0 80 F U Primary 1000 Good 0 0 0 81 M M Secondary 1000 Good 0 0 0 82 F M Secondary 1000 Good 0 0 0 83 M S Secondary 1000 Good 0 0 0 84 F S Secondary 1000 Good 0 0 0 85 M D Secondary 1000 Good 0 0 0 86 F D Secondary 1000 Good 0 0 0 87 M W Secondary 1000 Good 0 0 0 88 F W Secondary 1000 Good 0 0 0 89 M U Secondary 1000 Good 0 0 0 90 F U Secondary 1000 Good 0 0 0 91 M M College 1000 Good 0 0 0 92 F M College 1000 Good 0 0 0 93 M S College 1000 Good 0 0 0 94 F S College 1000 Good 0 0 0 95 M D College 1000 Good 0 0 0 96 F D College 1000 Good 0 0 0 97 M W College 1000 Good 0 0 0 98 F W College 1000 Good 0 0 0 99 M U College 1000 Good 0 0 0 100 F U College 1000 Good 0 0 0 101 M M 2 yr Degree 1000 Good 0 0 0 102 F M 2 yr Degree 1000 Good 0 0 0 103 M S 2 yr Degree 1000 Good 0 0 0 104 F S 2 yr Degree 1000 Good 0 0 0 105 M D 2 yr Degree 1000 Good 0 0 0 106 F D 2 yr Degree 1000 Good 0 0 0 107 M W 2 yr Degree 1000 Good 0 0 0 108 F W 2 yr Degree 1000 Good 0 0 0 109 M U 2 yr Degree 1000 Good 0 0 0 110 F U 2 yr Degree 1000 Good 0 0 0 111 M M 4 yr Degree 1000 Good 0 0 0 112 F M 4 yr Degree 1000 Good 0 0 0 113 M S 4 yr Degree 1000 Good 0 0 0 114 F S 4 yr Degree 1000 Good 0 0 0 115 M D 4 yr Degree 1000 Good 0 0 0 116 F D 4 yr Degree 1000 Good 0 0 0 117 M W 4 yr Degree 1000 Good 0 0 0 118 F W 4 yr 

### Joined the Customer Data with Demographics Data

In [0]:
joinExpr = customer_df.current_cdemo_sk == customer_demographics.cd_demo_sk
customerWithDemographics = customer_df.join(customer_demographics,joinExpr,"inner")

In [0]:
customerWithDemographics.select("customer_sk","firstName","lastName","cd_credit_rating").show(10)

+-----------+---------+--------+----------------+
customer_sk|firstName|lastName|cd_credit_rating|
+-----------+---------+--------+----------------+
 51826| Bernard| Wilson| Good|
 77500| Robert| Mullins| Good|
 45681| Hazel| Brown| Good|
 33313| Pauline| Slaton| Good|
 40692| Manuel| Muller| Good|
 27996| Kathleen| Thomas| Good|
 93526| Millie| Coley| Good|
 6359| Johnnie| Horton| Good|
 28095| Stanley| Roberts| Good|
 24147| Denise| Garcia| Good|
+-----------+---------+--------+----------------+
only showing top 10 rows

In [0]:
customerWithDemographics.select("cd_credit_rating").distinct().show()

+----------------+
cd_credit_rating|
+----------------+
 Low Risk|
 Good|
 Unknown|
 High Risk|
+----------------+

In [0]:
creditProfile = customerWithDemographics.groupBy("cd_credit_rating").count().withColumnRenamed("count","customerCount").withColumnRenamed("cd_credit_rating","creditRating")

display(creditProfile)

creditRating customerCount Low Risk 24133 Good 24038 Unknown 24009 High Risk 24382

In [0]:
customerPurchaseLevel = customerWithDemographics.groupBy("cd_purchase_estimate").count().withColumnRenamed("count","customerCount").withColumnRenamed("cd_purchase_estimate","purchaseLevelEstimate")
display(customerPurchaseLevel)

purchaseLevelEstimate customerCount 1500 4847 5000 4769 8500 4760 10000 4760 2000 4834 7000 4754 6500 4885 3000 4712 9500 4840 1000 4907 2500 4836 4000 4872 9000 4832 3500 4861 500 4877 7500 4746 4500 4903 5500 4885 6000 4793 8000 4889

In [0]:
customerPurchaseLevel.coalesce(1).write.mode("overwrite").option("header","true").csv("/Output/hbl/customerPurchaseLevel.csv")
customerPurchaseLevel.write.saveAsTable("customerPurchaseLevel")

In [0]:
# df_CPL = spark.read.option("header","true").csv("/FileStore/output/customerPurchaseLevel.csv")
# df_CPL.show(5)

### Let us run some predictive models to categorize the customers into `preferred`/`unpreferred` flag

In [0]:
df_model = customerWithDemographics.select("salutation","birth_Day","birth_Month","birth_Year","cd_gender","cd_marital_status","cd_education_status","cd_purchase_estimate","cd_credit_rating","cd_dep_count","cd_dep_employed_count","cd_dep_college_count","preferred_Cust_Flag")
df_model = df_model.filter(df_model.preferred_Cust_Flag.isNotNull())
df_model.write.saveAsTable("customerdata")
display(df_model)

salutation birth_Day birth_Month birth_Year cd_gender cd_marital_status cd_education_status cd_purchase_estimate cd_credit_rating cd_dep_count cd_dep_employed_count cd_dep_college_count preferred_Cust_Flag Sir 17 8 1936 M W Secondary 500 Good 0 0 0 N Sir 6 9 1926 F U Secondary 500 Good 0 0 0 Y Dr. 13 9 1928 M S 4 yr Degree 500 Good 0 0 0 Y Ms. 11 4 1992 F S 4 yr Degree 500 Good 0 0 0 N Mr. 11 7 1988 M M College 1000 Good 0 0 0 N Dr. 11 5 1948 F D 4 yr Degree 1000 Good 0 0 0 N Miss 30 6 1944 M M Advanced Degree 1000 Good 0 0 0 N Mr. 14 10 1934 M U Advanced Degree 1000 Good 0 0 0 N Dr. 28 4 1939 M M Unknown 1000 Good 0 0 0 N Ms. 23 5 1933 M S Primary 1500 Good 0 0 0 Y Sir 11 9 1932 M W College 1500 Good 0 0 0 N Dr. 27 3 1940 F M Advanced Degree 1500 Good 0 0 0 Y Mrs. 7 12 1925 M S Advanced Degree 1500 Good 0 0 0 Y Dr. 7 12 1950 F W Primary 2000 Good 0 0 0 N Ms. 1 8 1933 M D Unknown 2000 Good 0 0 0 N Sir 14 1 1942 F S Primary 2500 Good 0 0 0 N Sir 15 12 1933 M U Primary 2500 Good 0 0 0 N Ms. 28 3 1926 F U Primary 2500 Good 0 0 0 N Dr. 10 3 1952 F D College 2500 Good 0 0 0 Y Dr. 17 10 1965 M D 2 yr Degree 2500 Good 0 0 0 N Sir 22 2 1965 F M Primary 3000 Good 0 0 0 N Dr. 19 1 1933 M U Primary 3000 Good 0 0 0 N Dr. 20 1 1933 M D Secondary 3000 Good 0 0 0 Y Dr. 5 5 1966 M W Secondary 3000 Good 0 0 0 N Mr. 10 3 1969 F U Secondary 3000 Good 0 0 0 Y Dr. 6 5 1977 M M College 3000 Good 0 0 0 Y Miss 14 3 1988 M M 2 yr Degree 3000 Good 0 0 0 N Ms. 30 10 1955 F S 2 yr Degree 3000 Good 0 0 0 N Dr. 29 7 1930 M S 4 yr Degree 3000 Good 0 0 0 Y Miss 7 12 1974 M S 4 yr Degree 3500 Good 0 0 0 N Dr. 5 9 1970 F W Primary 4000 Good 0 0 0 N Dr. 20 9 1953 M S Secondary 4000 Good 0 0 0 N Dr. 6 2 1991 M W 2 yr Degree 4000 Good 0 0 0 N Mr. 6 10 1986 M S Advanced Degree 4000 Good 0 0 0 Y Mrs. 19 11 1983 M W Advanced Degree 4000 Good 0 0 0 N Sir 23 5 1924 M D Unknown 4000 Good 0 0 0 N Dr. 5 10 1977 M W College 4500 Good 0 0 0 Y Miss 18 1 1934 F U Advanced Degree 4500 Good 0 0 0 N Sir 13 7 1927 M W College 5000 Good 0 0 0 N Miss 23 4 1980 F D Advanced Degree 5000 Good 0 0 0 N Mrs. 9 1 1980 F S Primary 5500 Good 0 0 0 N Mr. 11 8 1986 F W Secondary 5500 Good 0 0 0 Y null null 4 null F W 2 yr Degree 5500 Good 0 0 0 N Mr. 11 8 1971 M M Advanced Degree 5500 Good 0 0 0 Y Dr. 22 7 1962 F M Unknown 5500 Good 0 0 0 Y Mr. 15 5 1982 M S College 6000 Good 0 0 0 N Dr. 11 8 1983 F S 4 yr Degree 6000 Good 0 0 0 Y Sir 5 6 1983 F U 4 yr Degree 6000 Good 0 0 0 Y Miss 3 2 1944 F S Primary 6500 Good 0 0 0 N Dr. 11 2 1959 M S College 6500 Good 0 0 0 Y Mrs. 17 8 1957 F S Advanced Degree 6500 Good 0 0 0 N Dr. 9 5 1958 M U Advanced Degree 6500 Good 0 0 0 N Dr. 22 9 1956 F D College 7000 Good 0 0 0 N Sir 27 7 1965 M S 2 yr Degree 7000 Good 0 0 0 N Miss 5 5 1990 M W 2 yr Degree 7000 Good 0 0 0 Y Dr. 3 2 1926 F U Secondary 7500 Good 0 0 0 Y Miss 19 3 1935 F S College 7500 Good 0 0 0 Y Miss 14 8 1937 F M Advanced Degree 7500 Good 0 0 0 N Dr. 6 12 1928 M D 4 yr Degree 8000 Good 0 0 0 N Ms. 22 3 1988 F M Advanced Degree 8000 Good 0 0 0 N Dr. 12 6 1973 M M Primary 8500 Good 0 0 0 Y Dr. 21 3 1991 M D Advanced Degree 8500 Good 0 0 0 N Miss 30 3 1988 M D Advanced Degree 8500 Good 0 0 0 Y Mrs. 25 4 1953 M D Secondary 9000 Good 0 0 0 Y Mr. 19 12 1974 F U 2 yr Degree 9000 Good 0 0 0 Y Miss 1 12 1947 M D Primary 9500 Good 0 0 0 N Mr. 17 10 1992 F U Secondary 9500 Good 0 0 0 Y Dr. 14 7 1936 M U College 9500 Good 0 0 0 Y Mr. 28 4 1924 F U 2 yr Degree 10000 Good 0 0 0 Y Sir 21 2 1959 M M Advanced Degree 10000 Good 0 0 0 N Dr. 27 11 1951 M S Advanced Degree 10000 Good 0 0 0 Y Sir 9 6 1932 M M College 500 Low Risk 0 0 0 N Mrs. 16 4 1989 M W College 500 Low Risk 0 0 0 N Dr. 24 6 1972 M M Unknown 500 Low Risk 0 0 0 N Sir 16 11 1937 M S Primary 1000 Low Risk 0 0 0 Y Dr. 15 12 1978 F D 4 yr Degree 1000 Low Risk 0 0 0 N Mrs. 1 3 1948 F W 4 yr Degree 1000 Low Risk 0 0 0 Y Mr. 19 6 1924 F M Secondary 2000 Low Risk 0 0 0 N Mrs. 28 7 1951 M M Advanced Degree 2000 Low Risk 0 0 0 N Mr. 30 1 1958 M D Secondary 25

In [0]:
cat_features = ["salutation","cd_gender","cd_marital_status","cd_education_status","cd_credit_rating"]
for i in cat_features:
  df_model.select(i).distinct().show()

+----------+
salutation|
+----------+
 Ms.|
 null|
 Mr.|
 Miss|
 Mrs.|
 Dr.|
 Sir|
+----------+

+---------+
cd_gender|
+---------+
 F|
 M|
+---------+

+-----------------+
cd_marital_status|
+-----------------+
 M|
 U|
 D|
 W|
 S|
+-----------------+

+-------------------+
cd_education_status|
+-------------------+
 4 yr Degree|
 Secondary|
 Unknown|
 Advanced Degree|
 Primary|
 College|
 2 yr Degree|
+-------------------+

+----------------+
cd_credit_rating|
+----------------+
 Low Risk|
 Good|
 Unknown|
 High Risk|
+----------------+

In [0]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer

for i in cat_features:
  stringindexer = StringIndexer(inputCol=i, outputCol=i+"_Num").setHandleInvalid("keep")  
  encoder = OneHotEncoder(inputCol=i+"_Num", outputCol=i+'_Vector')
  
  pipeline = Pipeline(stages=[stringindexer, encoder])
  pipeline_fit = pipeline.fit(df_model)
  df_model = pipeline_fit.transform(df_model)

In [0]:
display(df_model.head(5))

salutation birth_Day birth_Month birth_Year cd_gender cd_marital_status cd_education_status cd_purchase_estimate cd_credit_rating cd_dep_count cd_dep_employed_count cd_dep_college_count preferred_Cust_Flag salutation_Num salutation_Vector cd_gender_Num cd_gender_Vector cd_marital_status_Num cd_marital_status_Vector cd_education_status_Num cd_education_status_Vector cd_credit_rating_Num cd_credit_rating_Vector Sir 17 8 1936 M W Secondary 500 Good 0 0 0 N 2.0 List(0, 6, List(2), List(1.0)) 0.0 List(0, 2, List(0), List(1.0)) 2.0 List(0, 5, List(2), List(1.0)) 6.0 List(0, 7, List(6), List(1.0)) 3.0 List(0, 4, List(3), List(1.0)) Sir 6 9 1926 F U Secondary 500 Good 0 0 0 Y 2.0 List(0, 6, List(2), List(1.0)) 1.0 List(0, 2, List(1), List(1.0)) 1.0 List(0, 5, List(1), List(1.0)) 6.0 List(0, 7, List(6), List(1.0)) 3.0 List(0, 4, List(3), List(1.0)) Dr. 13 9 1928 M S 4 yr Degree 500 Good 0 0 0 Y 0.0 List(0, 6, List(0), List(1.0)) 0.0 List(0, 2, List(0), List(1.0)) 0.0 List(0, 5, List(0), List(1.0)) 3.0 List(0, 7, List(3), List(1.0)) 3.0 List(0, 4, List(3), List(1.0)) Ms. 11 4 1992 F S 4 yr Degree 500 Good 0 0 0 N 4.0 List(0, 6, List(4), List(1.0)) 1.0 List(0, 2, List(1), List(1.0)) 0.0 List(0, 5, List(0), List(1.0)) 3.0 List(0, 7, List(3), List(1.0)) 3.0 List(0, 4, List(3), List(1.0)) Mr. 11 7 1988 M M College 1000 Good 0 0 0 N 1.0 List(0, 6, List(1), List(1.0)) 0.0 List(0, 2, List(0), List(1.0)) 3.0 List(0, 5, List(3), List(1.0)) 4.0 List(0, 7, List(4), List(1.0)) 3.0 List(0, 4, List(3), List(1.0))

In [0]:
search_engine_indexer = StringIndexer(inputCol="preferred_Cust_Flag", outputCol="preferred_Cust_Flag_Num").setHandleInvalid("keep").fit(df_model)
df_model = search_engine_indexer.transform(df_model)

In [0]:
display(df_model.head(5))

salutation birth_Day birth_Month birth_Year cd_gender cd_marital_status cd_education_status cd_purchase_estimate cd_credit_rating cd_dep_count cd_dep_employed_count cd_dep_college_count preferred_Cust_Flag salutation_Num salutation_Vector cd_gender_Num cd_gender_Vector cd_marital_status_Num cd_marital_status_Vector cd_education_status_Num cd_education_status_Vector cd_credit_rating_Num cd_credit_rating_Vector preferred_Cust_Flag_Num Sir 17 8 1936 M W Secondary 500 Good 0 0 0 N 2.0 List(0, 6, List(2), List(1.0)) 0.0 List(0, 2, List(0), List(1.0)) 2.0 List(0, 5, List(2), List(1.0)) 6.0 List(0, 7, List(6), List(1.0)) 3.0 List(0, 4, List(3), List(1.0)) 0.0 Sir 6 9 1926 F U Secondary 500 Good 0 0 0 Y 2.0 List(0, 6, List(2), List(1.0)) 1.0 List(0, 2, List(1), List(1.0)) 1.0 List(0, 5, List(1), List(1.0)) 6.0 List(0, 7, List(6), List(1.0)) 3.0 List(0, 4, List(3), List(1.0)) 1.0 Dr. 13 9 1928 M S 4 yr Degree 500 Good 0 0 0 Y 0.0 List(0, 6, List(0), List(1.0)) 0.0 List(0, 2, List(0), List(1.0)) 0.0 List(0, 5, List(0), List(1.0)) 3.0 List(0, 7, List(3), List(1.0)) 3.0 List(0, 4, List(3), List(1.0)) 1.0 Ms. 11 4 1992 F S 4 yr Degree 500 Good 0 0 0 N 4.0 List(0, 6, List(4), List(1.0)) 1.0 List(0, 2, List(1), List(1.0)) 0.0 List(0, 5, List(0), List(1.0)) 3.0 List(0, 7, List(3), List(1.0)) 3.0 List(0, 4, List(3), List(1.0)) 0.0 Mr. 11 7 1988 M M College 1000 Good 0 0 0 N 1.0 List(0, 6, List(1), List(1.0)) 0.0 List(0, 2, List(0), List(1.0)) 3.0 List(0, 5, List(3), List(1.0)) 4.0 List(0, 7, List(4), List(1.0)) 3.0 List(0, 4, List(3), List(1.0)) 0.0

In [0]:
df_model = df_model.withColumn("cd_purchase_estimate",df_model["cd_purchase_estimate"].cast('integer'))
df_model = df_model.withColumn("preferred_Cust_Flag_Num",df_model["preferred_Cust_Flag_Num"].cast('integer'))
df_model = df_model.withColumn("birth_Day",df_model["birth_Day"].cast('integer'))
df_model = df_model.withColumn("birth_Month",df_model["birth_Month"].cast('integer'))
df_model = df_model.withColumn("birth_Year",df_model["birth_Year"].cast('integer'))

In [0]:
# 剔除冗余、不需要的字段
useful_cols = [
    # label目标值字段
    "preferred_Cust_Flag_Num",  
    
    # 特征值字段
#     "birth_Day", 
#     "birth_Month",
#     "birth_Year",
    "cd_purchase_estimate",
    "cd_dep_count",
    "cd_dep_employed_count",
    "cd_dep_college_count",
    "salutation_Vector",
    "cd_gender_Vector",
    "cd_marital_status_Vector",
    "cd_education_status_Vector",
    "cd_credit_rating_Vector"
]

df_model_select = df_model.select(*useful_cols)
df_model_select.printSchema()

root
-- preferred_Cust_Flag_Num: integer (nullable = true)
-- cd_purchase_estimate: integer (nullable = true)
-- cd_dep_count: integer (nullable = true)
-- cd_dep_employed_count: integer (nullable = true)
-- cd_dep_college_count: integer (nullable = true)
-- salutation_Vector: vector (nullable = true)
-- cd_gender_Vector: vector (nullable = true)
-- cd_marital_status_Vector: vector (nullable = true)
-- cd_education_status_Vector: vector (nullable = true)
-- cd_credit_rating_Vector: vector (nullable = true)

In [0]:
from pyspark.ml.feature import VectorAssembler  
 
df_model_select = df_model.select(*useful_cols)
df_model_select=df_model_select.dropna()

## 构成特征向量
df_assembler = VectorAssembler(inputCols=useful_cols[1:], outputCol="features")
df_model_select = df_assembler.transform(df_model_select)

df_model_select = df_model_select['features','preferred_Cust_Flag_Num']
 
# ## 查看构建后的数据
# model_df=df_model_select.select(['features','preferred_Cust_Flag_Num'])

In [0]:
training_df, test_df= df_model_select.randomSplit([0.7,0.3]) 

### We will apply several classification models to the model_df

In [0]:
from pyspark.ml.classification import LogisticRegression   

lr = LogisticRegression(labelCol='preferred_Cust_Flag_Num').fit(training_df)
# testresult = lr.transform(test_df)
print('{}{}'.format('LogisticRegression Train accuracy：',lr.evaluate(test_df).accuracy))

LogisticRegression Train accuracy：0.5063952062234993

In [0]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import StringIndexer, VectorIndexer,IndexToString

rf = RandomForestClassifier(labelCol="preferred_Cust_Flag_Num", \
                            featuresCol="features", \
                            numTrees = 50) 
#                             maxDepth = 4, \
#                             maxBins = 32)
# Train model with Training Data
rfModel = rf.fit(training_df)
predictions = rfModel.transform(test_df)

predictions.select("prediction", "preferred_Cust_Flag_Num", "features").show(5)

+----------+-----------------------+--------------------+
prediction|preferred_Cust_Flag_Num| features|
+----------+-----------------------+--------------------+
 0.0| 1|(28,[0,1,2,4,10,1...|
 0.0| 1|(28,[0,1,2,4,10,1...|
 0.0| 0|(28,[0,1,2,4,10,1...|
 0.0| 0|(28,[0,1,2,4,10,1...|
 0.0| 0|(28,[0,1,2,4,10,1...|
+----------+-----------------------+--------------------+
only showing top 5 rows

In [0]:
evaluator = MulticlassClassificationEvaluator(
    labelCol="preferred_Cust_Flag_Num", predictionCol="prediction", metricName="accuracy")

accuracy = evaluator.evaluate(predictions)
print("Test Error = %g" % (1.0 - accuracy))

Test Error = 0.491642

In [0]:
from itertools import chain
featureMetadata = chain(*predictions.schema["features"].metadata["ml_attr"]["attrs"].values())
attrs = sorted((attr["idx"], attr["name"]) for attr in featureMetadata)

name_list = []
fi_list=[]
for idx, name in attrs:
  name_list.append(name)
  fi_list.append(rfModel.featureImportances[idx])

In [0]:
import pandas as pd
feature_importances = pd.DataFrame(data = {'name':name_list, 'importance':fi_list})
display(feature_importances.sort_values('importance', ascending=False))

name importance cd_purchase_estimate 0.1692027424131199 cd_dep_count 0.11329974665859835 cd_dep_employed_count 0.10504114033957984 cd_dep_college_count 0.09073658888428261 salutation_Vector_Mr. 0.03309847043183216 cd_education_status_Vector_Advanced Degree 0.03251018838468351 cd_credit_rating_Vector_High Risk 0.02848969649120512 cd_marital_status_Vector_S 0.02845415461551753 cd_marital_status_Vector_D 0.025862784955305775 cd_credit_rating_Vector_Unknown 0.02560491322720338 cd_credit_rating_Vector_Low Risk 0.025559614137072705 cd_education_status_Vector_Secondary 0.02515574899807709 cd_marital_status_Vector_U 0.024117570362405145 cd_marital_status_Vector_M 0.022854819306391688 salutation_Vector_Dr. 0.02241092381708966 salutation_Vector_Mrs. 0.021475649335321558 salutation_Vector_Sir 0.02131619813960665 cd_marital_status_Vector_W 0.02017220251429517 salutation_Vector_Ms. 0.020096587626553083 salutation_Vector_Miss 0.019800446039324376 cd_gender_Vector_F 0.019016422746618532 cd_credit_rating_Vector_Good 0.018600245314767467 cd_gender_Vector_M 0.01749141843792454 cd_education_status_Vector_College 0.01633064657051486 cd_education_status_Vector_Primary 0.015319102807099517 cd_education_status_Vector_4 yr Degree 0.014592059855024478 cd_education_status_Vector_2 yr Degree 0.013736046075145524 cd_education_status_Vector_Unknown 0.009653871515439652

In [0]:
feature_importances_table = feature_importances.sort_values('importance', ascending=False)
spark_df = spark.createDataFrame(feature_importances_table)
spark_df.write.saveAsTable("hba_customer_featureImportance")

In [0]:
spark_df_table = spark.sql("select * from hba_customer_featureImportance")
display(spark_df_table)

name importance cd_dep_college_count 0.09073658888428261 salutation_Vector_Mr. 0.03309847043183216 cd_education_status_Vector_Advanced Degree 0.03251018838468351 cd_credit_rating_Vector_High Risk 0.02848969649120512 cd_education_status_Vector_Primary 0.015319102807099517 cd_education_status_Vector_4 yr Degree 0.014592059855024478 cd_education_status_Vector_2 yr Degree 0.013736046075145524 cd_education_status_Vector_Unknown 0.009653871515439652 cd_credit_rating_Vector_Low Risk 0.025559614137072705 cd_education_status_Vector_Secondary 0.02515574899807709 cd_marital_status_Vector_U 0.024117570362405145 cd_marital_status_Vector_M 0.022854819306391688 cd_marital_status_Vector_S 0.02845415461551753 cd_marital_status_Vector_D 0.025862784955305775 cd_credit_rating_Vector_Unknown 0.02560491322720338 cd_credit_rating_Vector_Good 0.018600245314767467 cd_gender_Vector_M 0.01749141843792454 cd_education_status_Vector_College 0.01633064657051486 cd_marital_status_Vector_W 0.02017220251429517 salutation_Vector_Ms. 0.020096587626553083 salutation_Vector_Miss 0.019800446039324376 cd_gender_Vector_F 0.019016422746618532 salutation_Vector_Dr. 0.02241092381708966 salutation_Vector_Mrs. 0.021475649335321558 salutation_Vector_Sir 0.02131619813960665 cd_purchase_estimate 0.1692027424131199 cd_dep_count 0.11329974665859835 cd_dep_employed_count 0.10504114033957984

### Connect to Azure ML Workspace

In [ ]:
from azureml.core import Workspace
from azureml.core.experiment import Experiment
from azureml.core.authentication import InteractiveLoginAuthentication

subscription_id = 'Your_Subscription_ID'
resource_group = 'herbalifedemo'
workspace_name = 'herbalifeAML'

interactive_auth = InteractiveLoginAuthentication(tenant_id = "Your_Tenant_ID", cloud = "AzureChinaCloud")
try:
    ws = Workspace(subscription_id=subscription_id, resource_group=resource_group,workspace_name=workspace_name, auth=interactive_auth) 
    print('Library configuration succeed')
except:
    print('Workspace not found')
    
experiment = Experiment(ws, "newPrediction")

### Set Compute Target

In [ ]:
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your cluster.
amlcompute_cluster_name = "hbltraincluster"

# Verify that cluster does not exit already.
try:
    compute_target = ComputerTarget(workspace=ws, name=amlcompute_cluster-name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.procisioning_configuration(vm_size='STANDARD_DS12_V2',
                                                           max_nodes=6)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

### MLFlow

In [0]:
import mlflow
import mlflow.spark
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.mllib.evaluation import BinaryClassificationMetrics
import mlflow

mlflow.set_experiment("/hbldatalake_mlflow")

def evalMetrics(summary):
  accuracy=summary.accuracy
  auc = summary.areaUnderROC
  return (accuracy, auc)

#begin pipeline run
with mlflow.start_run() as run:
  
  maxIter=100
  
  #split dataset into training and testing
  featureColumns = [
    # label目标值字段
    "preferred_Cust_Flag_Num",  
    # 特征值字段
    "birth_Day", 
    "birth_Month",
    "birth_Year",
    "cd_purchase_estimate",
    "cd_dep_count",
    "cd_dep_employed_count",
    "cd_dep_college_count",
    "salutation_Vector",
    "cd_gender_Vector",
    "cd_marital_status_Vector",
    "cd_education_status_Vector",
    "cd_credit_rating_Vector"
]
  (train,test) = df_model.select(*featureColumns).na.drop().randomSplit([0.8,0.2])
  ftCols = train.columns[1:]
  
  trainCount=train.count()
  testCount=test.count()
  mlflow.log_param("trainCount", trainCount)
  mlflow.log_param("testCount", testCount)
  
  assembler=VectorAssembler(inputCols=ftCols, outputCol="featureVector")
  scaler=StandardScaler(inputCol="featureVector",outputCol="features",withStd=True,withMean=False)
  lr=LogisticRegression(featuresCol="features",labelCol="preferred_Cust_Flag_Num",maxIter=100)
  pipeline=Pipeline(stages=[assembler,scaler,lr])
  
  #fit the model
  model=pipeline.fit(train)
  lrModel=model.stages[-1]
  
  #Get Metrics
  (accuracy, auc) = evalMetrics(lrModel.summary)
  
  #Log mlflow metrics for the LR model
  mlflow.log_metric("accuracy", accuracy)
  mlflow.log_metric("auc", auc)
  
  #log model generated
  mlflow.spark.log_model(model,"lrModel")

In [0]:
 mlflow.end_run()